In [ ]:
import os
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback, TensorBoard

In [ ]:
# Load the raw CIFAR-10 data
train_ds, test_ds = tfds.load("cifar10",
                              split=[tfds.Split.TRAIN, tfds.Split.TEST],
                              batch_size=-1)

In [ ]:
train_images = train_ds["image"]
test_images = test_ds["image"]
train_images.shape

In [ ]:
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

train_labels = train_ds["label"]
test_labels = test_ds["label"]
train_labels.shape

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

In [ ]:
print(train_images[0].numpy().min())
train_images[0].numpy().max()

In [ ]:
x_train = tf.dtypes.cast(train_images, tf.float32)#.astype('float32')
x_test = tf.dtypes.cast(test_images, tf.float32)
x_train /= 255
x_test /= 255

print(type(x_train))
print(x_train[0].numpy().min())
print(x_train[0].numpy().max())

print(type(x_test))
print(x_test[0].numpy().min())
print(x_test[0].numpy().max())

In [ ]:
# the use of batch normalization
# https://machinelearningmastery.com/how-to-accelerate-learning-of-deep-neural-networks-with-batch-normalization/
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(32, 32,3)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation= "softmax")
])

In [ ]:
sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(train_images, train_labels, epochs=5)

In [ ]:
# code from here https://keras.io/examples/cifar10_cnn/

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization

cnn_model = keras.Sequential([
    Conv2D(32, (3, 3), padding='same',
                 input_shape=train_images.shape[1:]),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(32, (3, 3)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(64, (3, 3)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512),
    BatchNormalization(),
    Dense(10),
    BatchNormalization(),
    Activation('softmax')
])

In [ ]:
sgd = keras.optimizers.SGD(lr=0.001, decay=1e-5, momentum=0.9, nesterov=True)

cnn_model.compile(optimizer=sgd,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn_model.summary()

In [ ]:
PATIENCE = 10
EPOCHS = 200
TENSORBOARD_LOGDIR = "../tensorboard/cifar-10/" + datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint = ModelCheckpoint("model-{val_accuracy:.2f}.h5", monitor="val_accuracy", verbose=1, save_best_only=True,
                             save_weights_only=True, mode="max")
early_stop = EarlyStopping(monitor="val_loss", patience=PATIENCE, mode="min")
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=PATIENCE, min_lr=1e-7, verbose=1, mode="min")

tensorboard_callback = TensorBoard(log_dir=TENSORBOARD_LOGDIR)
cnn_model.fit(x_train, train_labels,
              validation_data=(x_test, test_labels),
              callbacks=[checkpoint, reduce_lr, early_stop, tensorboard_callback],
              epochs=EPOCHS,
              verbose=0,
              shuffle=True,
              batch_size=32)

In [ ]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

In [ ]:
# Score trained model.
scores = cnn_model.evaluate(x_test, test_labels, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
# Confusion matrix result, 
# code from here: https://ermlab.com/en/blog/nlp/cifar-10-classification-using-keras-tutorial/

from sklearn.metrics import classification_report, confusion_matrix
Y_pred = cnn_model.predict(x_test, verbose=2)
y_pred = np.argmax(Y_pred, axis=1)

for ix in range(10):
    print(ix, confusion_matrix(test_labels,y_pred)[ix].sum())
cm = confusion_matrix(test_labels,y_pred)
print(cm)

# Visualizing of confusion matrix
import seaborn as sn
import pandas  as pd

df_cm = pd.DataFrame(cm, range(10),
                  range(10))
plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_cm, annot=True,annot_kws={"size": 12})# font size
plt.show()

In [ ]:
# code from here https://www.tensorflow.org/tutorials/keras/basic_classification
def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array[i], true_label[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, Y_pred, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, Y_pred,  test_labels)
plt.show()

In [ ]:
# Plot the first X test images, their predicted label, and the true label
# Color correct predictions in blue, incorrect predictions in red
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, Y_pred, test_labels, test_images)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, Y_pred, test_labels)
plt.show()